In [154]:
import torch.optim as optim
import importlib
import REBRNN
import torch
import pickle
importlib.reload(REBRNN)

<module 'REBRNN' from '/Users/tiril/Documents/IndividualProject/nuclear_repo/knowledge_extraction/relation_extraction/REBRNN.py'>

In [95]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model for NER 
model_name = 'dslim/distilbert-NER'

# Custom keywords
try:
    with open('keyword_matching/directory.pkl', 'rb') as file:
        keywords = pickle.load(file)
except FileNotFoundError or FileExistsError:
    with open('directory.pkl', 'rb') as file:
        keywords = pickle.load(file)

# Hyperparameters
input_size = 512
num_layers = 4          # may require tuning
hidden_size = 256       # may require tuning
num_classes = 97        # 96 different relations plus '0' for no relation
learning_rate = 0.001   # may require tuning
batch_size = 64
num_epochs = 5

In [155]:
model = REBRNN.RelationExtractorBRNN(
    input_size=input_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    num_classes=num_classes,
    batch_size=batch_size,
    model_name=model_name,
    device=device,
    custom_keywords=keywords,
    threshold=0.7
)

In [104]:
train_raw, _, test_raw, validation_raw = model.load_data(get_distant=False)

In [135]:
length = 10
train = model.preprocessor(train_raw, length=2*length)
test = model.preprocessor(test_raw, length=length)
validation = model.preprocessor(validation_raw, length=length)

model.set_data_loaders(train, validation, test)

Preprocessing took 0.08 minutes. 7 instances (35.00%) exceeded max length.
Preprocessing took 0.04 minutes. 2 instances (20.00%) exceeded max length.
Preprocessing took 0.04 minutes. 2 instances (20.00%) exceeded max length.


In [143]:
model.set_data_loaders(train, validation, test)

In [64]:
'''
max_length=512 and length=100:
Preprocessing took 0.59 minutes. 58 instances (29.00%) exceeded max length.
Preprocessing took 0.29 minutes. 36 instances (36.00%) exceeded max length.
Preprocessing took 0.29 minutes. 33 instances (33.00%) exceeded max length.

max_length=1024 and length=100:
Preprocessing took 0.60 minutes. 1 instances (0.50%) exceeded max length.
Preprocessing took 0.29 minutes. 0 instances (0.00%) exceeded max length.
Preprocessing took 0.30 minutes. 1 instances (1.00%) exceeded max length.
'''

Preprocessing took 0.92 minutes. 58 instances (29.00%) exceeded max length.
Preprocessing took 0.50 minutes. 36 instances (36.00%) exceeded max length.
Preprocessing took 0.46 minutes. 33 instances (33.00%) exceeded max length.


'\nmax_length=512 and length=100:\nPreprocessing took 0.59 minutes. 58 instances (29.00%) exceeded max length.\nPreprocessing took 0.29 minutes. 36 instances (36.00%) exceeded max length.\nPreprocessing took 0.29 minutes. 33 instances (33.00%) exceeded max length.\n\nmax_length=1024 and length=100:\nPreprocessing took 0.60 minutes. 1 instances (0.50%) exceeded max length.\nPreprocessing took 0.29 minutes. 0 instances (0.00%) exceeded max length.\nPreprocessing took 0.30 minutes. 1 instances (1.00%) exceeded max length.\n'

In [130]:
custom_loss_fn = REBRNN.CustomLoss(threshold=0.8)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [151]:
def train_model(model, custom_loss_fn, optimizer, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        total_train_loss = 0

        for batch in model.train_loader:
            print(batch['embeddings'].size())

        for batch in model.train_loader:
            optimizer.zero_grad()

            embeddings = batch['embeddings'].to(model.device)
            print(embeddings.size())
            entity_pairs = batch['entity_pairs']
            triplets = batch['labels']

            outputs = model(embeddings)
            loss = custom_loss_fn(entity_pairs, outputs, triplets)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
        
        avg_train_loss = total_train_loss / len(model.train_loader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss:.4f}")

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in model.val_loader:
                embeddings = batch['embeddings'].to(model.device)  # Use precomputed embeddings
                entity_pairs = batch['entity_pairs']
                triplets = batch['labels']

                outputs = model(embeddings)  # Pass embeddings to the model
                loss = custom_loss_fn(entity_pairs, outputs, triplets)
                total_val_loss += loss.item()
        
        avg_val_loss = total_val_loss / len(model.val_loader)
        print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}")
        
        model.train()

In [152]:
train_model(model, custom_loss_fn, optimizer, num_epochs=5)

torch.Size([20, 512, 768])
torch.Size([20, 512, 768])


RuntimeError: input.size(-1) must be equal to input_size. Expected 512, got 768

In [156]:
embeddings = torch.randn(20, 512, 768)
output = model.forward(embeddings)

RuntimeError: Expected hidden[0] size (8, 512, 256), got [8, 20, 256]